In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
from nicks_plot_utils import *
import numpy as np
from scipy.interpolate import interp1d
from scipy.integrate import quad, tplquad, nquad

from maid_interface import maid_2007_Npi  as maid
# from maid_interface_old import maid_Npi  as maid_old

In [2]:
w_bins = np.array([1.2, 1.22, 1.24, 1.26, 1.28, 1.3,
                   1.32, 1.34, 1.36, 1.38, 1.4, 1.42, 1.44, 1.46, 1.48, 1.5, 1.52,
                   1.54, 1.56, 1.58, 1.6, 1.62, 1.64, 1.66, 1.68, 1.7, 1.72, 1.74,
                   1.76, 1.78, 1.8])

q2_bins = np.array([1.1, 1.33, 1.56, 1.87, 2.23, 2.66, 3.5])

theta_bins = np.array([-1.0, -0.8, -0.6, -0.4, -0.2,
                       0.0, 0.2, 0.4, 0.6, 0.8])

phi_binning = 10

In [3]:
def get_maid_values(xs, w, q2, theta):
    ENERGY = 4.81726
    crossSections = []
    for phi in xs:
        crossSections.append(maid(ENERGY, w, q2, theta, np.degrees(phi)))
        
    return np.array(crossSections)


def maid_values(phi, w, q2, theta):
    ENERGY = 4.81726        
    return maid(ENERGY, w, q2, theta, np.degrees(phi))

def plot_maid_model(ax, w, q2, theta, xs, fmt='.'):
    # Get bin centers
    #_w = (w.left + w.right) / 2.0
    #_q2 = (q2.left + q2.right) / 2.0
    #_theta = (theta.left + theta.right) / 2.0
    # Get the cross section values from maid
    crossSections = get_maid_values(xs, w, q2, theta)
    # _ax = ax.twinx()
    ax.errorbar(xs, crossSections, fmt=fmt, alpha=0.8, ms=15)
    # ax.set_ylim(bottom=0, top=np.max(crossSections)*1.5)

    return np.array(crossSections)


def binCetnerCorrection(w, q2, theta, num_bins=10):
    # Get bin centers
    _w = (w['left']+ w['right']) / 2.0
    _q2 = (q2['left']+ q2['right']) / 2.0
    _theta = (theta['left']+ theta['right']) / 2.0

    theta_left = theta['left'] if theta['left'] != -1.0 else -0.99
    theta_right = theta['right'] if theta['right'] != 1.0 else 0.99
    
    # make a huge space of phis
    width = np.linspace(0,  2*np.pi, num_bins, endpoint=True)[1]
    left = np.array([i*width for i in range(num_bins+2)])
    right = np.array([(i-1)*width for i in range(num_bins+2)])

    center = (left+right)/2.0
    ys = []
    for _w_ in [w['left'], w['right']]:
        for _q2_ in [q2['left'], q2['right']]:
            for _theta_ in [theta_left, theta_right]:
                for xs in [left, right]:
                    # Get the cross section values from maid
                    crossSections = get_maid_values(xs, _w_, _q2_, _theta_)
                    ys.append(crossSections)

    crossSections_center = get_maid_values(center, _w, _q2, _theta)
    ys = np.array(ys)
    avg = np.sum(ys, axis=0)/ys.shape[0]

    error = np.sqrt(np.sum((ys - avg)**2,axis=0)/2)
    bin_center_corr = interp1d(center, avg/crossSections_center, kind='cubic')
    return (bin_center_corr, avg, crossSections_center, center, error)


def binCetnerCorrection2(w, q2, theta, num_bins=10):
    # Get bin centers
    _w = (w['left']+ w['right']) / 2.0
    _q2 = (q2['left']+ q2['right']) / 2.0
    _theta = (theta['left']+ theta['right']) / 2.0

    theta_left = theta['left'] if theta['left'] != -1.0 else -0.99
    theta_right = theta['right'] if theta['right'] != 1.0 else 0.99
    
    # make a huge space of phis
    width = np.linspace(0,  2*np.pi, num_bins, endpoint=True)[1]
    left = np.array([i*width for i in range(num_bins+2)])
    right = np.array([(i-1)*width for i in range(num_bins+2)])

    center = (left+right)/2.0
    ys = []

#     for xs in [left, right]:
#         # Get the cross section values from maid
#         crossSections = get_maid_values(xs, _w, _q2, _theta)
#         ys.append(crossSections)
    for i in range(num_bins):
        def bounds_phi(x, y, z):
            return [width*i, width*(i+1)]

        def bounds_w(x, y):
            return [w['left'], w['right']]

        def bounds_q2(x):
            return [q2['left'], q2['right']]

        def bounds_theta():
            return [theta['left'], theta['right']]
        
        x = nquad(maid_values, [bounds_phi, bounds_w, bounds_q2, bounds_theta], opts={'limit': 10})
        print(x[0])
        ys.append(x[0])
    crossSections_center = get_maid_values(center, _w, _q2, _theta)
    ys = np.array(ys)
    #avg = np.sum(ys, axis=0)/ys.shape[0]

    error = np.sqrt(np.sum((ys - avg)**2,axis=0)/2)
    bin_center_corr = interp1d(center, ys/crossSections_center, kind='cubic')
    return (bin_center_corr, avg, crossSections_center, center, error)






In [ ]:
for i, w in enumerate(w_bins[:-15]):
    for j, q2 in enumerate(q2_bins[:-3]):
        for theta in theta_bins[3:5]:
            fig, ax = plt.subplots(nrows=1, ncols=2, figsize=[16,8], sharex=True)
            xs = np.linspace(0,  2*np.pi, 100, endpoint=True)
            # phis = np.linspace(0,  2*np.pi, 10)
            W = {'left' : w, 'right' : w_bins[i+1]}
            Q2 = {'left' : q2, 'right' : q2_bins[j+1]}
            CT = {'left' : theta, 'right' : theta+0.2}
            
            _w = (W['left']+ W['right']) / 2.0
            _q2 = (Q2['left']+ Q2['right']) / 2.0
            _theta = (CT['left']+ CT['right']) / 2.0
            
            BC, avg, center, phis, error = binCetnerCorrection(W, Q2, CT)
            BC2, avg2, center2, phis, error = binCetnerCorrection2(W, Q2, CT)
            
            top = np.max(plot_maid_model(ax[0], _w, _q2, _theta, xs, fmt='-'))
            
            ax[1].plot(xs, BC(xs))
            ax[1].plot(xs, BC2(xs))
            ax[0].scatter(phis, center, label="center")
            ax[0].errorbar(phis, avg, fmt='.', xerr=phis[1], yerr=error)

            plt.title(f"$W$ = {w} $Q^2$ = {q2} $\\theta$ = {theta}")
            plt.xlim(0, 2*np.pi)
            ax[1].set_ylim(0, np.max(BC(xs))*1.3)
            ax[0].set_ylim(0, np.max(center)*1.3)
            plt.show()
            

/usr/local/lib/python3.8/site-packages/scipy/integrate/quadpack.py:880: IntegrationWarning: The maximum number of subdivisions (10) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  quad_r = quad(f, low, high, args=args, full_output=self.full_output,


In [ ]:
errors = []
for i, w in enumerate(w_bins[:-1]):
    for j, q2 in enumerate(q2_bins:-1):
        for theta in theta_bins:
            #fig, ax = plt.subplots(nrows=1, ncols=2, figsize=[16,8], sharex=True)
            xs = np.linspace(0,  2*np.pi, 100, endpoint=True)
            # phis = np.linspace(0,  2*np.pi, 10)
            W = {'left' : w, 'right' : w_bins[i+1]}
            Q2 = {'left' : q2, 'right' : q2_bins[j+1]}
            CT = {'left' : theta, 'right' : theta+0.2}
            
            _w = (W['left']+ W['right']) / 2.0
            _q2 = (Q2['left']+ Q2['right']) / 2.0
            _theta = (CT['left']+ CT['right']) / 2.0
            
            BC, avg, center, phis, error = binCetnerCorrection(W, Q2, CT)
            BC2, avg2, center2, phis, error = binCetnerCorrection2(W, Q2, CT)
            
            errors.append( np.sqrt( (BC(xs)-BC2(xs))**2 / 2))
            
            top = np.max(plot_maid_model(ax[0], _w, _q2, _theta, xs, fmt='-'))
            
            #ax[1].plot(xs, BC(xs))
            #ax[0].scatter(phis, center, label="center")
            #ax[0].errorbar(phis, avg, fmt='.', xerr=phis[1], yerr=error)

            #plt.title(f"$W$ = {w} $Q^2$ = {q2} $\\theta$ = {theta}")
            #plt.xlim(0, 2*np.pi)
            #ax[1].set_ylim(0, np.max(BC(xs))*1.3)
            #ax[0].set_ylim(0, np.max(center)*1.3)
            #plt.show()

In [ ]:
plt.hist(np.stack(errors, axis=1).flatten(), bins=100)
plt.show()